In [ ]:
!pip install transformers[torch] datasets pandas scikit-learn tqdm safetensors

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Program Files\Python313\python.exe -m pip install --upgrade pip


In [3]:
# ======================================================================
# 1단계: 기본 설정, 함수 및 클래스 정의
# ======================================================================
import os
import gc
from datetime import datetime
from pathlib import Path

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModel, Trainer, TrainingArguments, AutoConfig,
    EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm.auto import tqdm
import pytz # 시간대 설정을 위해 import

# --- 기본 설정 ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

MODEL_NAME = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
MAX_LEN = 400

# --- 경로 설정 (pathlib 사용) ---
# ❗️❗️ 경로를 환경에 맞게 수정해주세요 ❗️❗️
DRIVE_BASE_PATH = Path("/content/drive/MyDrive/Colab Notebooks")
CSV_FILE_PATH = DRIVE_BASE_PATH / "Labeled_Social_Data_Augmented_4Class.csv"
TEST_FILE_PATH = DRIVE_BASE_PATH / "TestFile.csv"

# --- 출력 폴더 설정 (한국 시간 기준) ---
KST = pytz.timezone('Asia/Seoul') # 한국 시간대 설정
RUN_ID = datetime.now(KST).strftime("%Y%m%d_%H%M%S") # KST 적용

BASE_OUT = Path("/content/drive/MyDrive/colab_results")
RUN_DIR = BASE_OUT / RUN_ID
OUTPUT_DIR_STAGE1 = RUN_DIR / "stage1_model"
OUTPUT_DIR_STAGE2 = RUN_DIR / "stage2_model"

# 폴더 생성
RUN_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR_STAGE1.mkdir(exist_ok=True)
OUTPUT_DIR_STAGE2.mkdir(exist_ok=True)
print(f"\n📁 해당 폴더 내에 저장됩니다 : {RUN_DIR}\n")


# --- 데이터 로더 클래스 ---
class DataCollatorForChunkedText:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, features: list) -> dict:
        all_chunks, all_labels, article_ids = [], [], []
        for i, feature in enumerate(features):
            all_chunks.extend(feature['content'])
            article_ids.extend([i] * len(feature['content']))
            all_labels.append(feature['labels'])

        inputs = self.tokenizer(
            all_chunks,
            padding="max_length",
            truncation=True,
            max_length=MAX_LEN,
            return_tensors="pt"
        )
        inputs['labels'] = torch.tensor(all_labels, dtype=torch.long)
        inputs['article_ids'] = torch.tensor(article_ids, dtype=torch.long)
        return inputs


# --- Attention 메커니즘이 적용된 커스텀 모델 클래스 ---
class CustomModelWithAttention(nn.Module):
    def __init__(self, model_name: str, num_labels: int):
        super().__init__()
        self.num_labels = num_labels
        cfg = AutoConfig.from_pretrained(
            model_name,
            hidden_dropout_prob=0.2,
            attention_probs_dropout_prob=0.2
        )
        self.bert = AutoModel.from_pretrained(model_name, config=cfg)
        self.attention = nn.Sequential(
            nn.Linear(cfg.hidden_size, 512),
            nn.Tanh(),
            nn.Dropout(0.1),
            nn.Linear(512, 1),
            nn.Softmax(dim=0)
        )
        self.classifier = nn.Linear(cfg.hidden_size, num_labels)
        self.class_weights = None
        self.label_smoothing = 0.05

    def forward(self, input_ids, attention_mask, article_ids, labels=None, **kwargs):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        chunk_embeddings = outputs.last_hidden_state[:, 0, :]

        num_articles = labels.shape[0] if labels is not None else article_ids.max() + 1
        final_article_embeddings = torch.zeros((num_articles, self.bert.config.hidden_size), device=chunk_embeddings.device)

        # 각 글(article)별로 청크를 처리하기 위해 for-loop 유지
        for i in range(num_articles):
            article_chunk_embeddings = chunk_embeddings[article_ids == i]
            if article_chunk_embeddings.shape[0] == 0: continue

            if article_chunk_embeddings.shape[0] == 1:
                final_article_embeddings[i] = article_chunk_embeddings.squeeze(0)
            else:
                weights = self.attention(article_chunk_embeddings)
                weighted_avg_embedding = torch.sum(article_chunk_embeddings * weights, dim=0)
                final_article_embeddings[i] = weighted_avg_embedding

        logits = self.classifier(final_article_embeddings)

        loss = None
        if labels is not None:
            weight = self.class_weights.to(logits.device) if self.class_weights is not None else None
            loss_fct = nn.CrossEntropyLoss(weight=weight, label_smoothing=self.label_smoothing)
            loss = loss_fct(logits, labels)

        return (loss, logits) if loss is not None else logits


# --- 성능 평가 함수 ---
def compute_metrics(eval_pred: tuple) -> dict:
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="macro", zero_division=0)

    print("\n--- Validation Report ---")
    print(classification_report(labels, preds, zero_division=0))
    print("-" * 25)
    return {"accuracy": acc, "macro_f1": f1}


# --- Trainer 생성 함수 ---
def make_trainer(df: pd.DataFrame, num_labels: int, output_dir: str, num_epochs: int = 5) -> Trainer:
    dataset = Dataset.from_pandas(df).train_test_split(test_size=0.2, seed=42)

    model = CustomModelWithAttention(MODEL_NAME, num_labels).to(device)

    # 클래스 가중치 계산
    labels_np = df["labels"].to_numpy(dtype="int64")
    class_counts = np.bincount(labels_np, minlength=num_labels)
    class_counts[class_counts == 0] = 1 # 0으로 나누는 것 방지
    class_weights = torch.tensor(len(labels_np) / (num_labels * class_counts), dtype=torch.float)
    model.class_weights = class_weights
    print(f"\nApplied Class Weights: {class_weights}\n")

    # TrainingArguments 설정
    args = TrainingArguments(
        output_dir=output_dir,
        save_strategy="epoch",
        eval_strategy="epoch",
        logging_strategy="steps",
        logging_steps=300,
        per_device_train_batch_size=4,
        num_train_epochs=num_epochs,
        learning_rate=1.5e-5,
        weight_decay=0.05,
        warmup_ratio=0.1,
        max_grad_norm=1.0,
        load_best_model_at_end=True,
        metric_for_best_model="macro_f1",
        greater_is_better=True,
        save_total_limit=1,
        fp16=torch.cuda.is_available(),
        report_to="none",
        remove_unused_columns=False,
        seed=42,
    )

    callbacks = [EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.0)]

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        data_collator=DataCollatorForChunkedText(tokenizer),
        compute_metrics=compute_metrics,
        callbacks=callbacks
    )
    return trainer

ModuleNotFoundError: No module named 'datasets'

In [ ]:
# ======================================================================
# 2단계: 데이터 전처리 및 학습셋 준비
# ======================================================================
df_original = pd.read_csv(CSV_FILE_PATH).reset_index(drop=True)


# 기본적인 결측치 및 중복 제거
df_original['content'] = df_original['content'].replace(r'^\s*$', np.nan, regex=True)
df_original = df_original.dropna(subset=['content', 'new_label']).reset_index(drop=True) # new_label 결측치도 제거
df_original = df_original.drop_duplicates(subset=['content'], keep='first').reset_index(drop=True)

# 유효한 4개 클래스 값만 필터링 (안전장치)
VALID4 = {'중립','찬성_개정강화','반대_현상유지','찬성_폐지완화'}
df_original = df_original[df_original['new_label'].isin(VALID4)].reset_index(drop=True)

print("\n=== 전체 new_label 분포(학습 포함 대상) ===")
print(df_original['new_label'].value_counts(), "\n")

# --- 슬라이딩 윈도우(청크) 함수 ---
def chunk_content(df):
    processed_data = []
    CHUNK_SIZE = MAX_LEN
    OVERLAP = 128
    STRIDE = CHUNK_SIZE - OVERLAP

    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Chunking..."):
        tokens = tokenizer.encode(row['content'], add_special_tokens=False)
        if len(tokens) > CHUNK_SIZE:
            content_list = [tokenizer.decode(tokens[i:i+CHUNK_SIZE]) for i in range(0, len(tokens), STRIDE)]
        else:
            content_list = [row['content']]
        processed_data.append({'content': content_list, 'labels': row['labels']})
    return pd.DataFrame(processed_data)

# --- 1단계 모델(3중 분류) 학습 데이터셋 ---
print("\n--- Preparing Data for Stage 1 Model (3-Class) ---")
label_map_s1 = {'찬성_개정강화': 0, '찬성_폐지완화': 1, '반대_현상유지': 1, '중립': 2}
df_stage1 = df_original.copy()
df_stage1['labels'] = df_stage1['new_label'].map(label_map_s1)
print("Stage 1 Label Distribution (articles):")
print(df_stage1['labels'].map({0:'긍정적', 1:'부정적', 2:'중립'}).value_counts(), "\n")
df_stage1_train = chunk_content(df_stage1[['content', 'labels']])

# --- 2단계 모델(2중 분류) 학습 데이터셋 ---
print("\n--- Preparing Data for Stage 2 Model (2-Class) ---")
label_map_s2 = {'찬성_폐지완화': 0, '반대_현상유지': 1}
df_stage2 = df_original[df_original['new_label'].isin(label_map_s2.keys())].copy()
df_stage2['labels'] = df_stage2['new_label'].map(label_map_s2)
print("Stage 2 Label Distribution (articles):")
print(df_stage2['labels'].map({0:'찬성_폐지완화', 1:'반대_현상유지'}).value_counts(), "\n")
df_stage2_train = chunk_content(df_stage2[['content', 'labels']])

print("\n✅ All datasets ready for training!")


=== 전체 new_label 분포(학습 포함 대상) ===
new_label
중립         6184
찬성_개정강화     345
반대_현상유지     255
찬성_폐지완화     119
Name: count, dtype: int64 


--- Preparing Data for Stage 1 Model (3-Class) ---
Stage 1 Label Distribution (articles):
labels
중립     6184
부정적     374
긍정적     345
Name: count, dtype: int64 



Chunking...:   0%|          | 0/6903 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors



--- Preparing Data for Stage 2 Model (2-Class) ---
Stage 2 Label Distribution (articles):
labels
반대_현상유지    255
찬성_폐지완화    119
Name: count, dtype: int64 



Chunking...:   0%|          | 0/374 [00:00<?, ?it/s]


✅ All datasets ready for training!


In [4]:
# ======================================================================
# 3단계: 1단계 모델 학습
# ======================================================================
print("\n--- Training Stage 1 Model (3-Class) ---")
trainer1 = make_trainer(df_stage1_train, num_labels=3, output_dir=OUTPUT_DIR_STAGE1, num_epochs=5)
trainer1.train()

# 학습 완료 후, Trainer가 자동으로 로드한 베스트 모델을 명시적으로 저장
BEST1_DIR = f"{RUN_DIR}/stage1_best_model"
trainer1.save_model(BEST1_DIR)
print(f"\n--- Stage 1 Best Model Saved at {BEST1_DIR} ---")


--- Training Stage 1 Model (3-Class) ---


pytorch_model.bin:   0%|          | 0.00/511M [00:00<?, ?B/s]


Applied Class Weights: tensor([6.6696, 6.1524, 0.3721])



model.safetensors:   0%|          | 0.00/511M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,1.326500,1.208327,0.928313,0.689650
2,1.248900,1.099105,0.905141,0.741696
3,1.233600,1.083078,0.950760,0.828299
4,1.060700,1.118681,0.946416,0.827027
5,1.088200,1.120520,0.941347,0.812917



--- Validation Report ---
              precision    recall  f1-score   support

           0       0.67      0.37      0.48        75
           1       0.77      0.53      0.63        70
           2       0.94      0.98      0.96      1236

    accuracy                           0.93      1381
   macro avg       0.79      0.63      0.69      1381
weighted avg       0.92      0.93      0.92      1381

-------------------------

--- Validation Report ---
              precision    recall  f1-score   support

           0       0.53      0.84      0.65        75
           1       0.50      0.84      0.62        70
           2       0.99      0.91      0.95      1236

    accuracy                           0.91      1381
   macro avg       0.67      0.87      0.74      1381
weighted avg       0.94      0.91      0.92      1381

-------------------------

--- Validation Report ---
              precision    recall  f1-score   support

           0       0.67      0.76      0.71       

In [5]:
# ======================================================================
# 4단계: 2단계 모델 학습
# ======================================================================
print("\n--- Training Stage 2 Model (2-Class) ---")
trainer2 = make_trainer(df_stage2_train, num_labels=2, output_dir=OUTPUT_DIR_STAGE2, num_epochs=5)
trainer2.train()

BEST2_DIR = f"{RUN_DIR}/stage2_best_model"
trainer2.save_model(BEST2_DIR)
print(f"\n--- Stage 2 Best Model Saved at {BEST2_DIR} ---")


--- Training Stage 2 Model (2-Class) ---

Applied Class Weights: tensor([1.5714, 0.7333])



Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,No log,0.594436,0.733333,0.710648
2,No log,0.396868,0.840000,0.838594
3,No log,0.283514,0.920000,0.917522
4,0.470100,0.255086,0.946667,0.944444
5,0.470100,0.257847,0.946667,0.944444



--- Validation Report ---
              precision    recall  f1-score   support

           0       0.68      0.59      0.63        29
           1       0.76      0.83      0.79        46

    accuracy                           0.73        75
   macro avg       0.72      0.71      0.71        75
weighted avg       0.73      0.73      0.73        75

-------------------------

--- Validation Report ---
              precision    recall  f1-score   support

           0       0.72      0.97      0.82        29
           1       0.97      0.76      0.85        46

    accuracy                           0.84        75
   macro avg       0.85      0.86      0.84        75
weighted avg       0.87      0.84      0.84        75

-------------------------

--- Validation Report ---
              precision    recall  f1-score   support

           0       0.85      0.97      0.90        29
           1       0.98      0.89      0.93        46

    accuracy                           0.92      

In [4]:
# ======================================================================
# 모델 로드에 필요한 필수 클래스 및 라이브러리 정의
# ======================================================================
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel, AutoConfig
import torch.nn.functional as F
import os

# --- 기본 설정 ---
# ⭐️⭐️ 이 변수들은 예측 함수 내부에서도 사용되므로 그대로 유지해야 합니다 ⭐️⭐️
MODEL_NAME = "beomi/KcELECTRA-base-v2022"
MAX_LEN = 400
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME) # 토크나이저도 로드해야 합니다.


# --- Attention 메커니즘이 적용된 커스텀 모델 클래스 정의 ---
# ⭐️⭐️ 이 부분이 없으면 모델을 불러올 수 없습니다 ⭐️⭐️
class CustomModelWithAttention(nn.Module):
    def __init__(self, model_name: str, num_labels: int):
        super().__init__()
        self.num_labels = num_labels
        cfg = AutoConfig.from_pretrained(
            model_name,
            hidden_dropout_prob=0.2,
            attention_probs_dropout_prob=0.2
        )
        self.bert = AutoModel.from_pretrained(model_name, config=cfg)
        self.attention = nn.Sequential(
            nn.Linear(cfg.hidden_size, 512),
            nn.Tanh(),
            nn.Dropout(0.1),
            nn.Linear(512, 1),
            nn.Softmax(dim=0)
        )
        self.classifier = nn.Linear(cfg.hidden_size, num_labels)
        self.class_weights = None
        self.label_smoothing = 0.05

    def forward(self, input_ids, attention_mask, article_ids=None, labels=None, **kwargs):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        chunk_embeddings = outputs.last_hidden_state[:, 0, :]

        # 예측 시에는 labels가 없으므로 article_ids 기준으로 article 개수를 정합니다.
        num_articles = article_ids.max().item() + 1
        final_article_embeddings = torch.zeros((num_articles, self.bert.config.hidden_size), device=chunk_embeddings.device)

        # 각 글(article)별로 청크를 처리
        for i in range(num_articles):
            article_chunk_embeddings = chunk_embeddings[article_ids == i]
            if article_chunk_embeddings.shape[0] == 0: continue

            if article_chunk_embeddings.shape[0] == 1:
                final_article_embeddings[i] = article_chunk_embeddings.squeeze(0)
            else:
                weights = self.attention(article_chunk_embeddings)
                weighted_avg_embedding = torch.sum(article_chunk_embeddings * weights, dim=0)
                final_article_embeddings[i] = weighted_avg_embedding

        logits = self.classifier(final_article_embeddings)

        loss = None
        # 예측 시에는 labels가 None이므로 이 부분은 실행되지 않습니다.
        if labels is not None:
            weight = self.class_weights.to(logits.device) if self.class_weights is not None else None
            loss_fct = nn.CrossEntropyLoss(weight=weight, label_smoothing=self.label_smoothing)
            loss = loss_fct(logits, labels)

        return (loss, logits) if loss is not None else logits

print("✅ 모델 로드에 필요한 클래스 및 설정이 완료되었습니다.")

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--beomi--KcELECTRA-base-v2022. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an 

✅ 모델 로드에 필요한 클래스 및 설정이 완료되었습니다.


In [5]:
# ======================================================================
# 5단계: 모델 로드 및 예측 함수 정의
# ======================================================================
from pathlib import Path
from safetensors.torch import load_file
import torch
import torch.nn.functional as F
import os

# --- 경로 수정 시작 ---
# 현재 .ipynb 파일이 있는 디렉토리를 기준으로 경로를 설정합니다.
# (이 코드는 .py 스크립트로 실행할 경우에도 동일하게 작동합니다)
current_dir = os.path.dirname(os.path.abspath("__file__")) # Jupyter Notebook에서는 이 방식이 더 안정적일 수 있습니다.
# 만약 위 코드가 경로를 제대로 못 잡으면, 아래 코드를 사용하세요.
# current_dir = '.' 

# 모델 가중치 파일의 전체 경로를 명확하게 지정합니다.
BEST1_MODEL_PATH = os.path.join(current_dir, "stage1_best_model", "model.safetensors")
BEST2_MODEL_PATH = os.path.join(current_dir, "stage2_best_model", "model.safetensors")
# --- 경로 수정 끝 ---

# ----------------------------------------------------

print(f"\nLoading best trained models...")
try:
    # 1단계 모델 로드
    loaded_model1 = CustomModelWithAttention(MODEL_NAME, num_labels=3)
    # 수정된 경로 변수 사용
    state_dict1 = load_file(BEST1_MODEL_PATH, device="cpu")
    loaded_model1.load_state_dict(state_dict1)
    loaded_model1.to(device)
    loaded_model1.eval()

    # 2단계 모델 로드
    loaded_model2 = CustomModelWithAttention(MODEL_NAME, num_labels=2)
    # 수정된 경로 변수 사용
    state_dict2 = load_file(BEST2_MODEL_PATH, device="cpu")
    loaded_model2.load_state_dict(state_dict2)
    loaded_model2.to(device)
    loaded_model2.eval()

    print("✅ Best models loaded successfully!")

except Exception as e:
    print(f"❌ Model loading failed: {e}")
    print("아래 경로에 'model.safetensors' 파일이 있는지, 경로가 올바른지 확인해주세요:")
    # 오류 메시지도 수정된 경로로 표시
    print(f"  - 1단계 모델 경로: {BEST1_MODEL_PATH}")
    print(f"  - 2단계 모델 경로: {BEST2_MODEL_PATH}")

# --- 예측 함수 정의 ---
def predict_two_stage_revised(text, model1, model2, tokenizer, device):
    CHUNK_SIZE = MAX_LEN
    OVERLAP = 128
    STRIDE = CHUNK_SIZE - OVERLAP

    tokens = tokenizer.encode(text, add_special_tokens=False)
    if len(tokens) < CHUNK_SIZE:
        content_list = [text]
    else:
        content_list = [tokenizer.decode(tokens[i:i+CHUNK_SIZE]) for i in range(0, len(tokens), STRIDE)]

    inputs = tokenizer(content_list, padding="max_length", truncation=True, max_length=MAX_LEN, return_tensors="pt")
    inputs['article_ids'] = torch.zeros(len(content_list), dtype=torch.long)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        # 1단계 예측
        _, logits1 = model1(**inputs, labels=torch.tensor([0]).to(device))
        probs1 = F.softmax(logits1, dim=-1).squeeze()
        prob_positive = probs1[0].item()
        prob_negative = probs1[1].item()
        prob_neutral  = probs1[2].item()

        # 2단계 예측
        _, logits2 = model2(**inputs, labels=torch.tensor([0]).to(device))
        probs2  = F.softmax(logits2, dim=-1).squeeze()
        prob_repeal_given_negative   = probs2[0].item()
        prob_maintain_given_negative = probs2[1].item()

    # 최종 확률 계산
    final_probabilities = {
        '찬성_개정강화': prob_positive,
        '찬성_폐지완화': prob_negative * prob_repeal_given_negative,
        '반대_현상유지': prob_negative * prob_maintain_given_negative,
        '중립': prob_neutral
    }

    # 정규화 (전체 합이 1이 되도록)
    total_prob = sum(final_probabilities.values())
    if total_prob > 0:
        for k in final_probabilities:
            final_probabilities[k] = final_probabilities[k] / total_prob

    final_label = max(final_probabilities, key=final_probabilities.get)
    return final_label, final_probabilities


Loading best trained models...


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


✅ Best models loaded successfully!


In [ ]:
# ======================================================================
# 6단계: 샘플 테스트
# ======================================================================
samples = [
    # === [예상: 찬성_개정강화] ===
    "미래 세대를 위해 이 법은 반드시 필요합니다. 조속한 도입을 촉구합니다.",
    "솜방망이 처벌로는 부족합니다. 처벌 수위를 높이는 방향으로 법을 개정해야 합니다.",
    "이번에 발의된 동물보호법 개정안은 매우 시의적절한 조치라고 생각하며, 조속한 통과를 강력히 지지합니다.",
    "처벌 규정을 지금보다 더 강화하지 않는다면 피해자는 늘어날 수밖에 없다.",
    "혹시라도 이 개정안을 반대한다면 결국 범죄자 편을 드는 셈 아닌가?",
    "강화된 법안이 부담스럽더라도 장기적으로는 반드시 통과되어야 한다.",
    "나는 법을 좋아하지 않지만 이번만큼은 제정이 꼭 필요하다고 본다.",
    "바꾸지 않는 것이 낫다고 말하는 건 결국 현 상황을 방치하겠다는 뜻일 뿐이다.",

    # === [예상: 찬성_폐지완화] ===
    "시대에 뒤떨어진 이 규제는 당장 폐지해야 합니다.",
    "기업 활동을 위축시키는 불필요한 법은 완화하는 것이 맞습니다.",
    "더 이상 실효성도 없는 이 법을 유지할 이유가 없습니다. 즉각 철폐를 요구합니다.",
    "규제가 과도하다면 차라리 없애는 게 맞지 않겠는가?",
    "이 법을 유지하는 게 오히려 국민에게 손해라면 더 늦기 전에 폐지해야 한다.",
    "안전을 명분으로 한 규제 강화는 과하다, 최소한 완화는 필요하다.",
    "현실과 맞지 않는 조항은 더 이상 두지 말고 정리해야 한다.",
    "강화하지 않아도 충분하다면 불필요한 규제는 줄이는 게 낫다.",

    # === [예상: 반대_현상유지] ===
    "이번 개정안은 졸속으로 처리해서는 안 됩니다. 결사반대합니다.",
    "현행법을 무력화하려는 시도를 즉각 중단하십시오.",
    "섣부른 법 개정은 더 큰 혼란만 야기할 뿐입니다. 현상 유지가 최선입니다.",
    "결사반대입니다. 지금 법안 그래도 유지되어야 합니다.",
    "개정하면 좋아질 것 같지만 실제로는 행정 부담만 늘 뿐이다.",
    "없애자는 주장도 있지만 그렇게 하면 혼란이 더 커진다.",
    "나는 법안이 잘못됐다고 보지 않는다, 건드리면 악화될 뿐이다.",
    "개정? 제정? 오히려 지금 틀을 건드리지 않는 게 안전하다.",
    "모두가 고쳐야 한다고 하지만, 그대로 두는 편이 최선일 수도 있다.",

    # === [예상: 중립] ===
    "해당 법안은 어제 국무회의를 통과했으며, 국회 상정을 앞두고 있습니다.",
    "그래서 구체적으로 어떤 부분이 바뀌는 건가요?",
    "찬성 측은 제도의 실효성을 강조하는 반면, 반대 측은 개인의 자유 침해를 우려하고 있습니다.",
    "주 4일 근무제 도입에 대한 논의가 활발합니다. 찬성 측은 노동자의 '워라밸' 향상을, 반대 측은 기업의 인건비 부담 증가를 우려하고 있습니다."
    "이 법안은 현재 국회에 계류 중이다.",
    "찬성과 반대 의견이 모두 제시된 상태다.",
    "해외에서는 유사한 제도가 시행 중이다.",
    "국민청원 참여자는 약 5만 명에 달했다.",
    "법률 전문가들 사이에서도 합의가 이루어지지 않았다."
]

# 모델 로드가 성공적으로 완료되었는지 확인 후 테스트 진행
if 'loaded_model1' in locals() and 'loaded_model2' in locals():
    print("\n--- Revised 2-Stage Model Prediction Test ---")

    # tqdm을 사용하여 예측 진행 상황 표시
    results = []
    for s in tqdm(samples, desc="Predicting..."):
        label, probs = predict_two_stage_revised(s, loaded_model1, loaded_model2, tokenizer, device)
        results.append((s, label, probs))

    # 예측이 모두 끝난 후 결과 출력
    for text, label, probs in results:
        # 확률을 내림차순으로 정렬하여 문자열로 변환
        sorted_probs = sorted(probs.items(), key=lambda item: item[1], reverse=True)
        prob_str = " | ".join([f"{k}: {v*100:.2f}%" for k, v in sorted_probs])

        print("-" * 70)
        print(f'"{text[:70]}..."')
        print(f"➡️ Predicted Label: {label}")
        print(f"   - Probabilities: {prob_str}")
    print("-" * 70)

else:
    print("\n--- Skipping Prediction Test: Models were not loaded successfully. ---")


--- Revised 2-Stage Model Prediction Test ---


Predicting...:   0%|          | 0/33 [00:00<?, ?it/s]

----------------------------------------------------------------------
"미래 세대를 위해 이 법은 반드시 필요합니다. 조속한 도입을 촉구합니다...."
➡️ Predicted Label: 찬성_개정강화
   - Probabilities: 찬성_개정강화: 98.98% | 찬성_폐지완화: 0.80% | 중립: 0.21% | 반대_현상유지: 0.01%
----------------------------------------------------------------------
"솜방망이 처벌로는 부족합니다. 처벌 수위를 높이는 방향으로 법을 개정해야 합니다...."
➡️ Predicted Label: 찬성_개정강화
   - Probabilities: 찬성_개정강화: 98.94% | 찬성_폐지완화: 0.81% | 중립: 0.24% | 반대_현상유지: 0.01%
----------------------------------------------------------------------
"이번에 발의된 동물보호법 개정안은 매우 시의적절한 조치라고 생각하며, 조속한 통과를 강력히 지지합니다...."
➡️ Predicted Label: 찬성_개정강화
   - Probabilities: 찬성_개정강화: 99.13% | 반대_현상유지: 0.45% | 중립: 0.22% | 찬성_폐지완화: 0.20%
----------------------------------------------------------------------
"처벌 규정을 지금보다 더 강화하지 않는다면 피해자는 늘어날 수밖에 없다...."
➡️ Predicted Label: 찬성_개정강화
   - Probabilities: 찬성_개정강화: 98.89% | 찬성_폐지완화: 0.85% | 중립: 0.25% | 반대_현상유지: 0.01%
----------------------------------------------------------------------

In [ ]:
# ======================================================================
# 7단계: 외부 테스트 파일로 성능 평가
# ======================================================================
print("\n--- Evaluating on External Test File ---")

import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix
import json 

VALID4 = {'찬성_개정강화', '찬성_폐지완화', '반대_현상유지', '중립'}

# ---------- 유틸리티 함수들 ----------
def norm_label_for_eval(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().strip("[]'\"").strip()
    s = s.replace(" ", "").replace("-", "_").replace("__", "_")
    if s in VALID4: return s
    if s in {"찬성", "반대", "중립"}: return s
    mapping = {'찬성개정강화': '찬성_개정강화', '찬성폐지완화': '찬성_폐지완화', '반대현상유지': '반대_현상유지'}
    if s in mapping: return mapping[s]
    if "찬성" in s: return "찬성"
    if "반대" in s: return "반대"
    if "중립" in s: return "중립"
    return np.nan

def collapse4_to3(lbl):
    if lbl in {'찬성_개정강화', '찬성_폐지완화'}: return '찬성'
    if lbl == '반대_현상유지': return '반대'
    return '중립' if lbl == '중립' else np.nan

def norm_level(x):
    if pd.isna(x): return np.nan
    return str(x).strip().strip("[]'\"").strip()

def print_confusion(cm, labels):
    max_label_len = max(len(lbl) for lbl in labels) if labels else 10
    cell_width = max(max_label_len, 10)
    header = f"{'True':>{max_label_len}} \\ {'Predicted':^{cell_width * len(labels)}}"
    header_labels = "".join([f"{lbl:^{cell_width}}" for lbl in labels])
    tqdm.write("\n" + header)
    tqdm.write(f"{'':>{max_label_len}}   {header_labels}")
    tqdm.write("-" * (len(header) + len(labels)))
    for i, row in enumerate(cm):
        line = f"{labels[i]:>{max_label_len}} | " + "".join([f"{n:^{cell_width}d}" for n in row])
        tqdm.write(line)

# ---------- 테스트 파일 로드 및 전처리 ----------
try:
    df_test_raw = pd.read_csv(TEST_FILE_PATH, encoding="cp949")
except UnicodeDecodeError:
    df_test_raw = pd.read_csv(TEST_FILE_PATH, encoding="utf-8")

if 'content' not in df_test_raw.columns or 'label' not in df_test_raw.columns:
    raise KeyError("❌ 테스트 파일에는 반드시 'content'와 'label' 컬럼이 있어야 합니다.")

df_test = df_test_raw.copy()
df_test['content'] = df_test['content'].astype(str).replace(r'^\s*$', np.nan, regex=True)
df_test = df_test.dropna(subset=['content']).reset_index(drop=True)
df_test['label_raw'] = df_test['label']
df_test['label_std'] = df_test['label'].apply(norm_label_for_eval)
df_test = df_test.dropna(subset=['label_std']).reset_index(drop=True)

has_level = 'level' in df_test.columns
if has_level:
    df_test['level_raw'] = df_test['level']
    df_test['level_std'] = df_test['level'].apply(norm_level)

# ---------- 예측 및 개별 결과 출력 ----------
pred_labels_4 = []
probs_list = []
for i, row in tqdm(df_test.iterrows(), total=len(df_test), desc="Testing..."):
    txt = row['content']
    pred4, prob_dict = predict_two_stage_revised(txt, loaded_model1, loaded_model2, tokenizer, device)
    pred_labels_4.append(pred4)
    probs_list.append(prob_dict)

    # tqdm.write를 사용하여 진행 바 유지하며 출력
    output_str = "\n" + "="*90 + "\n"
    output_str += f"[샘플 {i+1}]\n"
    output_str += f"본문: {txt[:160]}{'...' if len(txt)>160 else ''}\n"

    #  정답 및 예측 결과 출력
    output_str += f"실제 정답: {row['label_raw']}\n"
    if has_level:
        output_str += f"난이도 level: {row.get('level_raw', 'N/A')}\n"

    output_str += f"예측 결과: {pred4}\n"
    output_str += "예측 확률:\n"
    for k, v in sorted(prob_dict.items(), key=lambda item: item[1], reverse=True):
        output_str += f"   - {k}: {v:.4f}\n"
    tqdm.write(output_str.strip())

df_test['pred_4class'] = pred_labels_4

# ======================================================================
# 7-2단계: 종합 지표 평가 및 결과 저장
# ======================================================================
# (이하 코드는 변경 없음)
print("\n\n" + "="*90)
print(" " * 35 + "FINAL EVALUATION REPORT")
print("="*90)

# ---------- 전체 지표 ----------
print("\n--- Metrics (Overall) ---")
is_4_class_ground_truth = all(label in VALID4 for label in df_test['label_std'].unique())
df_test['pred_3class'] = df_test['pred_4class'].apply(collapse4_to3)
df_test['label_3class'] = df_test['label_std'].apply(collapse4_to3)
metrics = {}

if is_4_class_ground_truth:
    print("\n[4-클래스 평가]")
    y_true, y_pred = df_test['label_std'], df_test['pred_4class']
    report = classification_report(y_true, y_pred, digits=4, zero_division=0)
    print(report)
    acc = accuracy_score(y_true, y_pred)
    macro_f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    print(f"Accuracy: {acc:.4f} | Macro F1: {macro_f1:.4f}")
    metrics = {"mode":"4-class", "accuracy":acc, "macro_f1":macro_f1}
else:
    print(f"\n[3-클래스 평가]")
    y_true, y_pred = df_test['label_3class'], df_test['pred_3class']
    valid_mask = y_true.notna() & y_pred.notna()
    if valid_mask.sum() > 0:
        report = classification_report(y_true[valid_mask], y_pred[valid_mask], digits=4, zero_division=0)
        print(report)
        acc = accuracy_score(y_true[valid_mask], y_pred[valid_mask])
        macro_f1 = f1_score(y_true[valid_mask], y_pred[valid_mask], average='macro', zero_division=0)
        print(f"Accuracy: {acc:.4f} | Macro F1: {macro_f1:.4f}")
        metrics = {"mode":"3-class", "accuracy":acc, "macro_f1":macro_f1}
    else:
        print("⚠️ 유효한 3클래스 라벨이 없어 평가를 생략합니다.")

# ---------- 레벨(난이도)별 지표 ----------
if has_level:
    print("\n--- Metrics by Level ---")
    levels = sorted(df_test['level_std'].dropna().unique())
    for lv in levels:
        sub_df = df_test[df_test['level_std'] == lv].copy()
        print(f"\n[Level = {lv}] (n={len(sub_df)})")
        if is_4_class_ground_truth:
            y_true_lv, y_pred_lv = sub_df['label_std'], sub_df['pred_4class']
            labels_order = sorted(list(VALID4))
            print("[4-클래스 평가]")
            print(classification_report(y_true_lv, y_pred_lv, labels=labels_order, digits=4, zero_division=0))
            cm = confusion_matrix(y_true_lv, y_pred_lv, labels=labels_order)
            print("Confusion Matrix (True \\ Predicted):")
            cm_df = pd.DataFrame(cm, index=[f"True_{l}" for l in labels_order], columns=[f"Pred_{l}" for l in labels_order])
            print(cm_df)
        else:
            y_true_lv, y_pred_lv = sub_df['label_3class'], sub_df['pred_3class']
            valid_mask_lv = y_true_lv.notna() & y_pred_lv.notna()
            labels_order_3 = sorted(['찬성', '반대', '중립'])
            print("[3-클래스 평가]")
            if valid_mask_lv.sum() > 0:
                print(classification_report(y_true_lv[valid_mask_lv], y_pred_lv[valid_mask_lv], labels=labels_order_3, digits=4, zero_division=0))
                cm = confusion_matrix(y_true_lv[valid_mask_lv], y_pred_lv[valid_mask_lv], labels=labels_order_3)
                print("Confusion Matrix (True \\ Predicted):")
                cm_df = pd.DataFrame(cm, index=[f"True_{l}" for l in labels_order_3], columns=[f"Pred_{l}" for l in labels_order_3])
                print(cm_df)
            else:
                print("⚠️ 유효한 3클래스 라벨이 없어 평가를 생략합니다.")
else:
    print("\nℹ️ 'level' 컬럼이 없어 레벨별 지표는 생략했습니다.")

# ---------- 최종 결과 파일 저장 ----------
if metrics.get("mode") == "4-class":
    df_test['is_correct'] = (df_test['label_std'] == df_test['pred_4class'])
elif metrics.get("mode") == "3-class":
    df_test['is_correct'] = (df_test['label_3class'] == df_test['pred_3class'])

prob_df = pd.DataFrame(probs_list)
df_test = pd.concat([df_test, prob_df.add_prefix('prob_')], axis=1)

first_cols = ['content', 'label_raw', 'label_std', 'pred_4class', 'is_correct']
if has_level:
    first_cols.append('level_std')
other_cols = [c for c in df_test.columns if c not in first_cols]
df_test = df_test[first_cols + other_cols]

PRED_CSV_PATH = f"{RUN_DIR}/test_predictions_{RUN_ID}.csv"
df_test.to_csv(PRED_CSV_PATH, index=False, encoding="utf-8-sig")
with open(f"{RUN_DIR}/test_metrics_{RUN_ID}.json","w", encoding="utf-8") as f:
    json.dump(metrics, f, ensure_ascii=False, indent=2)

print("\n" + "="*90)
print(f"\n✅ Saved predictions to: {PRED_CSV_PATH}")
print(f"✅ Saved metrics json to: {RUN_DIR}/test_metrics_{RUN_ID}.json")
print("\n📊 Final Metrics:", metrics)
print("\n" + "="*90)


--- Evaluating on External Test File ---


Testing...:   2%|▏         | 2/96 [00:00<00:08, 11.63it/s]

[샘플 1]
본문: 사회 안전망에 존재하는 심각한 허점을 보완하기 위해 이번 개정안은 반드시 필요합니다. 현행법으로는 급변하는 범죄 양상을 따라갈 수 없습니다. 처벌 조항을 대폭 상향하고, 법의 적용 범위를 넓혀서 국민이 안심하고 살 수 있는 환경을 만들어야 합니다. 조속한 법안 통과를 강력히 지지합니다.
실제 정답: [찬성_개정강화]
난이도 level: 기본
예측 결과: 찬성_개정강화
예측 확률:
   - 찬성_개정강화: 0.9895
   - 찬성_폐지완화: 0.0079
   - 중립: 0.0025
   - 반대_현상유지: 0.0001
[샘플 2]
본문: 최근 발생한 일련의 사건들은 현행법의 한계를 명확히 보여주었습니다. 지금의 솜방망이 처벌로는 범죄 예방 효과를 기대하기 어렵습니다. 국민의 법 감정에 부응하고 정의를 실현하기 위해, 관련 처벌 규정을 지금보다 훨씬 더 엄격하게 강화하는 방향으로 법을 개정할 것을 강력히 촉구합니다.
실제 정답: [찬성_개정강화]
난이도 level: 기본
예측 결과: 찬성_개정강화
예측 확률:
   - 찬성_개정강화: 0.9900
   - 찬성_폐지완화: 0.0075
   - 중립: 0.0024
   - 반대_현상유지: 0.0001
[샘플 3]
본문: 피해자는 있는데 가해자에 대한 처벌은 미미한 것이 현실입니다. 이는 법이 제 역할을 다하지 못하고 있다는 증거입니다. 피해자 보호를 최우선 가치로 두고, 가해자에게는 그 죄에 상응하는 무거운 책임을 물을 수 있도록 법 조항을 더욱 강력하게 손질해야 합니다. 더 이상의 관용은 없어야 합니...
실제 정답: [찬성_개정강화]
난이도 level: 기본
예측 결과: 찬성_개정강화
예측 확률:
   - 찬성_개정강화: 0.9892
   - 찬성_폐지완화: 0.0081
   - 중립: 0.0026
   - 반대_현상유지: 0.0001


Testing...:   6%|▋         | 6/96 [00:00<00:06, 14.79it/s]

[샘플 4]
본문: 이 법의 실효성을 높이기 위한 가장 확실한 방법은 처벌 수위를 현실화하는 것입니다. 현재의 벌금액과 형량은 범죄를 통해 얻는 이익에 비해 너무나도 가볍습니다. 범죄를 저지를 엄두조차 내지 못하도록, 징벌적 손해배상액을 높이고 형량을 대폭 상향하는 내용의 개정이 시급합니다.
실제 정답: [찬성_개정강화]
난이도 level: 기본
예측 결과: 찬성_개정강화
예측 확률:
   - 찬성_개정강화: 0.9899
   - 찬성_폐지완화: 0.0076
   - 중립: 0.0024
   - 반대_현상유지: 0.0001
[샘플 5]
본문: 국민의 생명과 안전보다 더 중요한 가치는 없습니다. 산업 현장에서 반복되는 비극을 막기 위해서는 기업의 안전 의무를 법적으로 더욱 강하게 규정해야 합니다. 안전 규정을 위반했을 경우, 현재보다 훨씬 더 강력한 행정적, 형사적 책임을 부과하는 방향으로 법을 강화해야 합니다.
실제 정답: [찬성_개정강화]
난이도 level: 기본
예측 결과: 찬성_개정강화
예측 확률:
   - 찬성_개정강화: 0.9901
   - 찬성_폐지완화: 0.0073
   - 중립: 0.0024
   - 반대_현상유지: 0.0001
[샘플 6]
본문: 이 법은 제정 당시의 목적을 이미 상실했으며, 지금은 오히려 경제 성장의 발목을 잡는 족쇄로 전락했습니다. 불필요한 행정 절차와 과도한 규제로 인해 수많은 기업이 고통받고 있습니다. 이제는 과감하게 이 법을 폐지하여 시장에 활력을 불어넣고, 기업들이 자유롭게 경쟁할 수 있는 환경을 만들...
실제 정답: [찬성_폐지완화]
난이도 level: 기본
예측 결과: 찬성_폐지완화
예측 확률:
   - 찬성_폐지완화: 0.9818
   - 찬성_개정강화: 0.0113
   - 반대_현상유지: 0.0059
   - 중립: 0.0010
[샘플 7]
본문: 기술은 하루가 다르게 발전하는데, 법은 수십 년 전에 머물러 있습니다. 현실과 동떨어진 이 낡은 규제는 혁신을 가로막는 가장 큰 걸림돌입니다. 미래 세대에게 더 

...

[샘플 88]
본문: 지난달 국회 상임위원회를 통과한 OO법 개정안이 오늘부터 국회 법제사법위원회의 본격적인 심사 절차에 들어갔다. 해당 개정안은 기존 법률의 제15조 3항에 명시된 처벌 규정을 강화하고, 적용 대상을 기존의 50인 이상 사업장에서 5인 이상 사업장으로 확대하는 내용을 골자로 하고 있다. 법...
실제 정답: 중립
난이도 level: 기본(긴글)
예측 결과: 중립
예측 확률:
   - 중립: 0.6268
   - 찬성_개정강화: 0.1820
   - 반대_현상유지: 0.1654
   - 찬성_폐지완화: 0.0258
[샘플 89]
본문: 정부는 오늘 오전 경제관계장관회의를 열고, 최근 과열 양상을 보이는 부동산 시장을 안정시키기 위한 새로운 종합 대책을 발표했다. 이번 대책에는 다주택자에 대한 종합부동산세율 인상, 투기과열지구 내 주택담보대출 비율(LTV) 축소, 그리고 청년 및 신혼부부를 위한 공공주택 공급 확대 방안...
실제 정답: 중립
난이도 level: 기본(긴글)
예측 결과: 중립
예측 확률:
   - 중립: 0.5074
   - 찬성_개정강화: 0.3604
   - 찬성_폐지완화: 0.0924
   - 반대_현상유지: 0.0399
[샘플 90]
본문: 현재 논란이 되고 있는 OO법은 1988년에 처음 제정되었다. 제정 당시 이 법은 급격한 산업화 과정에서 발생하는 환경오염 문제를 해결하고, 국민의 건강권을 보호하는 것을 주된 목적으로 삼았다. 이후 1995년 1차 개정을 통해 규제 대상 물질의 범위가 확대되었고, 2008년 2차 개정...
실제 정답: 중립
난이도 level: 기본(긴글)
예측 결과: 중립
예측 확률:
   - 중립: 0.6302
   - 찬성_폐지완화: 0.1838
   - 찬성_개정강화: 0.1785
   - 반대_현상유지: 0.0074
[샘플 91]
본문: 이번 개정안의 핵심은 '표현의 자유'와 '피해자 보호'라는 두 가지 가치가 충돌한다는 데 있다. 찬성 측은 온라인 공간에서의 혐오 표현과 가짜뉴스가 사회적 흉

Testing...:  98%|█████████▊| 94/96 [00:06<00:00, 16.15it/s]

[샘플 92]
본문: 최근 정부가 발표한 OO 정책을 두고 전문가들 사이에서 갑론을박이 이어지고 있다. 정책의 실효성과 지속가능성에 대한 의문이 주된 쟁점이다. 경제학자 A씨는 "정책의 취지는 좋으나, 막대한 재원 조달 계획이 불투명하여 장기적으로 국가 재정에 큰 부담을 줄 수 있다"는 문제점을 지적했다. ...
실제 정답: 중립
난이도 level: 중간(긴글)
예측 결과: 중립
예측 확률:
   - 중립: 0.6470
   - 찬성_개정강화: 0.1819
   - 반대_현상유지: 0.1380
   - 찬성_폐지완화: 0.0331
[샘플 93]
본문: 우리가 현재 논의하고 있는 이 법안과 유사한 제도를 먼저 도입했던 해외 국가들의 사례는 우리에게 많은 시사점을 준다. A국의 경우, 제도 도입 초기 강력한 규제로 인해 관련 산업이 위축되는 부작용을 겪었으나, 이후 규제를 완화하고 인센티브를 제공하는 방향으로 선회하면서 시장이 다시 활성...
실제 정답: 중립
난이도 level: 중간(긴글)
예측 결과: 중립
예측 확률:
   - 중립: 0.6418
   - 찬성_개정강화: 0.1800
   - 찬성_폐지완화: 0.1566
   - 반대_현상유지: 0.0215
[샘플 94]
본문: 기대를 모았던 OO법 개정안 공청회가 결국 양측의 격렬한 몸싸움과 고성으로 얼룩지며 파행으로 끝났다. 시민단체 측 토론자는 "이번 개정안은 국민의 기본권을 심각하게?침해하고, 민주주의를 후퇴시키는?악법"이라며 원색적인?비난을 쏟아냈다. 그는 법안이 통과될 경우 발생할 사회적?재앙에 대해...
실제 정답: 중립
난이도 level: 까다로움(긴글)
예측 결과: 중립
예측 확률:
   - 중립: 0.6517
   - 찬성_개정강화: 0.1767
   - 반대_현상유지: 0.1583
   - 찬성_폐지완화: 0.0133
[샘플 95]
본문: 감사원이 오늘 발표한 'OO사업 추진 실태'에 대한 감사 결과 보고서는 충격적이다. 보고서는 해당 사업이 초기 계획 단계부터 얼마나 부실하게 설계되었는지를

Testing...: 100%|██████████| 96/96 [00:06<00:00, 15.67it/s]


[샘플 96]
본문: 이번 대규모 정보 유출 사태는 해당 기업의 보안 시스템이 얼마나 취약했는지를 명백히 보여주었다. 사측은 "예측 불가능한 해커의 공격"이었다고 변명했지만, 전문가들은 "기본적인 보안 수칙조차 지키지 않은 명백한?인재(人災)"라고?비판의 목소리를 높이고 있다. 수백만 명의 개인정보가 유출되...
실제 정답: 중립
난이도 level: 까다로움(긴글)
예측 결과: 중립
예측 확률:
   - 중립: 0.6537
   - 찬성_개정강화: 0.1773
   - 반대_현상유지: 0.0896
   - 찬성_폐지완화: 0.0794


                                   FINAL EVALUATION REPORT

--- Metrics (Overall) ---

[4-클래스 평가]
              precision    recall  f1-score   support

     반대_현상유지     0.8889    1.0000    0.9412        24
          중립     1.0000    0.9167    0.9565        24
     찬성_개정강화     1.0000    0.9167    0.9565        24
     찬성_폐지완화     0.9200    0.9583    0.9388        24

    accuracy                         0.9479        96
   macro avg     0.9522    0.9479    0.9482        96
weighted avg     0.9522    0.9479    0.9482        96

Accuracy: 0.9479 | Macro F1: 0.9482

--- Metrics by Level ---

[Level = 기본] (n=20)
[4-클래스 평가]
              precision    recall  f1-score   support

     반대_현상유지  

In [6]:
import pandas as pd
from tqdm import tqdm
import torch

# --- CSV 파일 경로 ---
CSV_FILE_PATH = "news_comments_all.csv"
OUTPUT_CSV = "news_comments_all_감성분석완료.csv"

# --- CSV 불러오기 ---
try:
    df = pd.read_csv(CSV_FILE_PATH, encoding="utf-8")
    print(f"✅ '{CSV_FILE_PATH}' 파일 로딩 완료. (총 {len(df)}개 데이터)")
except FileNotFoundError:
    print(f"❌ 오류: '{CSV_FILE_PATH}' 파일을 찾을 수 없습니다. 파일 이름과 경로를 확인해주세요.")
    # 파일이 없으면 프로그램을 종료합니다.
    exit()

# --- GPU 사용 가능 여부 확인 및 device 설정 ---
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"✅ GPU를 사용합니다. (장치: {torch.cuda.get_device_name(0)})")
else:
    device = torch.device("cpu")
    print("✅ GPU를 사용할 수 없어 CPU를 사용합니다.")

# --- (중요) 모델과 토크나이저를 해당 device로 이동 ---
# 이 코드를 실행하기 전에 모델 로드 셀이 먼저 실행되어 있어야 합니다.
try:
    loaded_model1.to(device)
    loaded_model2.to(device)
    print("✅ 모델을 성공적으로 device로 이동했습니다.")
except NameError:
    print("❌ 오류: 'loaded_model1' 또는 'loaded_model2'가 정의되지 않았습니다.")
    print("   모델 예측 셀을 실행하기 전에, 반드시 모델 로드 셀을 먼저 실행해주세요.")
    exit()

# --- 예측 수행 ---
results = [] # 결과를 저장할 리스트

total_rows = len(df)
print(f"\n--- 전체 데이터 {total_rows}개에 대해 예측을 시작합니다 ---")

# df['content']에 NaN 값이 있을 경우를 대비해 문자열로 변환
for text in tqdm(df['content'].astype(str), desc="Predicting...", total=total_rows):
    try:
        label, probs = predict_two_stage_revised(text, loaded_model1, loaded_model2, tokenizer, device)
        results.append({
            'label_4class': label,
            'prob_찬성_개정강화': probs.get('찬성_개정강화', 0),
            'prob_찬성_폐지완화': probs.get('찬성_폐지완화', 0),
            'prob_반대_현상유지': probs.get('반대_현상유지', 0),
            'prob_중립': probs.get('중립', 0)
        })
    except Exception as e:
        results.append({
            'label_4class': "예측실패",
            'prob_찬성_개정강화': None,
            'prob_찬성_폐지완화': None,
            'prob_반대_현상유지': None,
            'prob_중립': None
        })
        # 너무 많은 오류 메시지가 출력되는 것을 방지하기 위해 주석 처리할 수 있습니다.
        # print(f"❌ 예측 실패 (텍스트: '{str(text)[:50]}...'): {e}")

# --- 결과 데이터프레임 생성 및 기존 데이터프레임과 병합 ---
results_df = pd.DataFrame(results)

# 기존 df와 결과 df를 옆으로 합칩니다.
# 두 데이터프레임의 인덱스가 동일하므로, reset_index() 없이 바로 합칠 수 있습니다.
df = pd.concat([df, results_df], axis=1)


# --- 라벨 단순화 작업 ---
print("\n--- 라벨을 '찬성', '반대', '중립'으로 단순화합니다 ---")

def simplify_label(label):
    if isinstance(label, str):
        if '찬성' in label:
            return '찬성'
        elif '반대' in label:
            return '반대'
        # '중립'이나 '예측실패' 등 다른 경우는 원래 값을 그대로 반환합니다.
        else:
            return label
    # 문자열이 아닌 경우 (예: NaN) 원래 값을 반환합니다.
    return label

# 'label_simplified' 라는 새로운 열을 만듭니다.
df['label_simplified'] = df['label_4class'].apply(simplify_label)

# 변환 후 라벨 종류 및 개수 확인
print("✅ 라벨 단순화 완료. 최종 라벨 분포:")
print(df['label_simplified'].value_counts())

# --- 저장 ---
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print(f"\n✅ 저장 완료: {OUTPUT_CSV}")

✅ 'news_comments_all.csv' 파일 로딩 완료. (총 48529개 데이터)
✅ GPU를 사용합니다. (장치: NVIDIA GeForce RTX 2070)
✅ 모델을 성공적으로 device로 이동했습니다.

--- 전체 데이터 48529개에 대해 예측을 시작합니다 ---


Predicting...:   1%|          | 503/48529 [00:22<36:10, 22.13it/s] 


KeyboardInterrupt: 

In [7]:
import pandas as pd
from tqdm import tqdm
import torch

# --- CSV 파일 경로 ---
CSV_FILE_PATH = "news_comments_all.csv"
OUTPUT_CSV = "news_comments_all_감성분석완료.csv"

# --- CSV 불러오기 ---
try:
    df = pd.read_csv(CSV_FILE_PATH, encoding="utf-8")
    print(f"✅ '{CSV_FILE_PATH}' 파일 로딩 완료. (총 {len(df)}개 데이터)")
except FileNotFoundError:
    print(f"❌ 오류: '{CSV_FILE_PATH}' 파일을 찾을 수 없습니다. 파일 이름과 경로를 확인해주세요.")
    # 파일이 없으면 프로그램을 종료합니다.
    exit()

# --- GPU 사용 가능 여부 확인 및 device 설정 ---
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"✅ GPU를 사용합니다. (장치: {torch.cuda.get_device_name(0)})")
else:
    device = torch.device("cpu")
    print("✅ GPU를 사용할 수 없어 CPU를 사용합니다.")

# --- (중요) 모델과 토크나이저를 해당 device로 이동 ---
# 이 코드를 실행하기 전에 모델 로드 셀이 먼저 실행되어 있어야 합니다.
try:
    loaded_model1.to(device)
    loaded_model2.to(device)
    print("✅ 모델을 성공적으로 device로 이동했습니다.")
except NameError:
    print("❌ 오류: 'loaded_model1' 또는 'loaded_model2'가 정의되지 않았습니다.")
    print("   모델 예측 셀을 실행하기 전에, 반드시 모델 로드 셀을 먼저 실행해주세요.")
    exit()

# --- 예측 수행 ---
results = [] # 결과를 저장할 리스트

total_rows = len(df)
print(f"\n--- 전체 데이터 {total_rows}개에 대해 예측을 시작합니다 ---")

# df['content']에 NaN 값이 있을 경우를 대비해 문자열로 변환
for text in tqdm(df['content'].astype(str), desc="Predicting...", total=total_rows):
    try:
        label, probs = predict_two_stage_revised(text, loaded_model1, loaded_model2, tokenizer, device)
        results.append({
            'label_4class': label, # 모델이 예측한 4가지 라벨
            'prob_찬성_개정강화': probs.get('찬성_개정강화', 0),
            'prob_찬성_폐지완화': probs.get('찬성_폐지완화', 0),
            'prob_반대_현상유지': probs.get('반대_현상유지', 0),
            'prob_중립': probs.get('중립', 0)
        })
    except Exception as e:
        results.append({
            'label_4class': "예측실패",
            'prob_찬성_개정강화': None,
            'prob_찬성_폐지완화': None,
            'prob_반대_현상유지': None,
            'prob_중립': None
        })
        # 너무 많은 오류 메시지가 출력되는 것을 방지하기 위해 주석 처리할 수 있습니다.
        # print(f"❌ 예측 실패 (텍스트: '{str(text)[:50]}...'): {e}")

# --- 결과 데이터프레임 생성 및 기존 데이터프레임과 병합 ---
results_df = pd.DataFrame(results)

# 기존 df와 결과 df를 옆으로 합칩니다.
df = pd.concat([df, results_df], axis=1)

# --- (제거됨) 라벨 단순화 작업 ---
# print("\n--- 라벨을 '찬성', '반대', '중립'으로 단순화합니다 ---")
# ... (simplify_label 함수 및 apply 로직 전체 삭제) ...

# --- 최종 라벨 분포 확인 ---
print("\n✅ 예측 완료. 최종 라벨 분포:")
# 'label_simplified' 대신 'label_4class' 열의 분포를 확인합니다.
print(df['label_4class'].value_counts())

# --- 저장 ---
# 열 이름을 'label_4class'에서 더 일반적인 'label'로 변경하여 저장할 수 있습니다.
df.rename(columns={'label_4class': 'label'}, inplace=True)
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print(f"\n✅ 저장 완료: {OUTPUT_CSV}")

✅ 'news_comments_all.csv' 파일 로딩 완료. (총 48529개 데이터)
✅ GPU를 사용합니다. (장치: NVIDIA GeForce RTX 2070)
✅ 모델을 성공적으로 device로 이동했습니다.

--- 전체 데이터 48529개에 대해 예측을 시작합니다 ---


Predicting...: 100%|██████████| 48529/48529 [36:29<00:00, 22.17it/s]



✅ 예측 완료. 최종 라벨 분포:
label_4class
중립         33397
찬성_개정강화     6573
반대_현상유지     5786
찬성_폐지완화     2773
Name: count, dtype: int64

✅ 저장 완료: news_comments_all_감성분석완료.csv


In [11]:
import pandas as pd
pd.set_option('display.max_rows', None)  # 모든 행을 출력하도록 설정
pd.set_option('display.max_columns', None)  # 모든 열을 출력하도록 설정
pd.set_option('display.width', None)  # 너비에 맞춰 줄 바꿈이 일어나지 않도록 설정
pd.set_option('display.max_colwidth', None) # 열 내용이 생략되지 않도록 설정

social_df = pd.read_csv("news_comments_all_감성분석완료.csv", encoding="utf-8")

# pd.crosstab을 사용하여 cluster_event_name별 label 개수 계산
label_counts_df = pd.crosstab(index=social_df['cluster_event_name'], columns=social_df['label'])

# 4개의 label이 항상 열에 포함되도록 보정 (만약 특정 label이 데이터에 없을 경우를 대비)
all_labels = ['찬성_개정강화', '찬성_폐지완화', '반대_현상유지', '중립', '예측실패']
for label in all_labels:
    if label not in label_counts_df.columns:
        label_counts_df[label] = 0

# 원하는 순서대로 열 정렬
label_counts_df = label_counts_df[all_labels]

# 결과 출력
print("--- cluster_event_name 별 label 개수 ---")
print(label_counts_df)

--- cluster_event_name 별 label 개수 ---
label                         찬성_개정강화  찬성_폐지완화  반대_현상유지    중립  예측실패
cluster_event_name                                                 
12·3 국회 계엄군 결박 사건                   2        0        0     3     0
12·3 비상계엄 기자 포박 사건                  0        0        2    38     0
12·3 비상계엄 내란 수사 사건                  0        0        0     5     0
2023 대선 후보 첫 TV토론                   1        2       19    55     0
2023년 개인정보 유출 해킹 사건                 2        2        0     3     0
2024년 양도소득세 시행령 개정안                 2        1        3     3     0
2024년 연말정산 보험료 세액공제                 0        0        0     2     0
2025 범금융 신년인사회 경제 안정화 노력            2        2        5    17     0
2025 에스원 AI CCTV 보안 트렌드             1        0        0     4     0
2025년 최악 살인기업 아리셀 사건                1        1        2     1     0
30대 미성년자 성착취물 제작 사건                 1        0        0     4     0
500대 기업 여성 임원 비율 증가 사건              4        5        2    31     0
8조 종투사 IMA